Importing required modules

In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas

Reading the dataset after it has been processed and cleaned.

In [2]:
df = pd.read_csv("cleanData.csv")

In [3]:
df.head()

,year,area,production,productivity,caneCrushed,sugar,sugarRecovery,sugarFactories,state
0,2000.0,4316.0,4316.0,4316.0,4316.0,4316.0,4316.0,4316.0,allOverIndia
1,2001.0,4411.0,4411.0,4411.0,4411.0,4411.0,4411.0,4411.0,allOverIndia
2,2002.0,4520.0,4520.0,4520.0,4520.0,4520.0,4520.0,4520.0,allOverIndia
3,2003.0,3938.0,3938.0,3938.0,3938.0,3938.0,3938.0,3938.0,allOverIndia
4,2004.0,3662.0,3662.0,3662.0,3662.0,3662.0,3662.0,3662.0,allOverIndia


In [4]:
df.columns

Index(['year', 'area', 'production', 'productivity', 'caneCrushed', 'sugar',
       'sugarRecovery', 'sugarFactories', 'state'],
      dtype='object')

Forming a new interactive dataframe for the dashboard

In [5]:
idf = df.interactive()

Year Slider

In [6]:
year_slider = pn.widgets.IntSlider(name = "Year slider", start = 1950, end = 2021, step = 2 )

In [7]:
year_slider

IntSlider(end=2021, name='Year slider', start=1950, step=2, value=1950)

Getting names of all states 

In [8]:
df['state'].unique()

array(['allOverIndia', 'andhraPradesh', 'assam', 'bihar', 'chhattisgarh',
       'gujrat', 'haryana', 'karnataka', 'kerala', 'maha', 'orrisa',
       'punjab', 'rajasthan', 'tamilNadu', 'telangana', 'uttarPradesh',
       'uttarakhand', 'westBengal'], dtype=object)

In [9]:
states = ['andhraPradesh', 'assam', 'bihar', 'chhattisgarh','gujrat', 'haryana', 'karnataka', 'kerala', 'maha', 'orrisa','punjab', 'rajasthan', 'tamilNadu', 'telangana', 'uttarPradesh','uttarakhand', 'westBengal']

Plot 1  - Line plot of sugar, sugarFactories and sugarRecovery over the years by states

X axis - Year
Y axis - The attribute which is being plot

In [10]:
yaxis_sugar = pn.widgets.RadioButtonGroup(name = 'Y axis',
                                         options = ['sugar','sugarFactories','sugarRecovery'],
                                         button_type = 'success')

setting a stream or pipeline of data which only filters the sugar, sugarFactories and sugarRecovery attributes by year and state.

In [11]:
sugarPipeline = ( 
    idf[
        (idf.year <= year_slider) & (idf.state.isin(states))
    ]
    .groupby(['state','year'])[yaxis_sugar].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = "year")
    .reset_index(drop=True)


)


connecting the pipeline and plot in notebook

In [12]:
sugar_plot = sugarPipeline.hvplot(x = 'year',by = 'state', y = yaxis_sugar,line_width=2,title = "Sugar by state")

In [13]:
sugar_plot

using the same pipepline preparing a table

In [14]:
sugar_table = sugarPipeline.pipe(pn.widgets.Tabulator,pagination ='remote',page_size = 10, sizing_mode = 'stretch_width')

In [15]:
sugar_table

Plot 2 - Scatter plot - Area vs Production 

Creating a pipeline for data used in scatter plot - area vs production

In [16]:
area_vs_production_scatterplot_pipeline = (
    idf [
        ((idf.year == year_slider) & (idf.state.isin(states)))
        
    ]
    .groupby(['state','year','production'])['area'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = "year")
    .reset_index(drop=True)
)

In [17]:
area_vs_production_scatterplot_pipeline

Connecting pipeline and notebook using scatterplot 

In [18]:
area_vs_production_scatterplot = area_vs_production_scatterplot_pipeline.hvplot(x = 'production',y = 'area',by='state',size=80,kind='scatter',alpha=0.7,legend=False,height=500,width=500)

In [19]:
area_vs_production_scatterplot

Plot 3 - Production and productivity over the years by states

X axis - Year
Y axis - The attribute which is being plot

In [20]:
yaxis_production = pn.widgets.RadioButtonGroup(name = 'Y axis',
                                         options = ['production','productivity'],
                                         button_type = 'success')

setting a stream or pipeline of data which only filters the sugar, sugarFactories and sugarRecovery attributes by year and state.

In [21]:
productionPipeline = ( 
    idf[
        (idf.year <= year_slider) & (idf.state.isin(states))
    ]
    .groupby(['state','year'])[yaxis_production].mean()
    .to_frame()
    .reset_index()
    .sort_values(by = "year")
    .reset_index(drop=True)


)


In [22]:
productionPipeline

connecting the pipeline and plot in notebook

In [23]:
production_plot = productionPipeline.hvplot(x = 'year',by = 'state', y = yaxis_production,line_width=2,title = "Production by state")

In [24]:
production_plot

In [25]:
yaxis_caneCrushed = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['caneCrushed','area'],
    button_type = 'success'
)

In [26]:
caneCrushed_bar_pipeline = (
    idf[
        (idf.year == year_slider) & (idf.state.isin(states))
    ]
    .groupby(['state','year'])[yaxis_caneCrushed].sum()
    .to_frame()
    .reset_index()
    .sort_values(by = "year")
    .reset_index(drop=True)
    
)

In [27]:
caneCrushed_bar_plot = caneCrushed_bar_pipeline.hvplot(kind="barh",x = "state", y = yaxis_caneCrushed , title = "caneCrushed and Area")

In [28]:
caneCrushed_bar_plot

In [30]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Sugarcane Statistics India', 
    sidebar=[pn.pane.Markdown("###India is the second-largest sugarcane producer globally after Brazil. In India, 21% of agricultural area is used for sugarcane production. The country produces about 22% of the total sugarcane in the world. The largest producer of sugarcane in India is the state of Uttar Pradesh.The second largest producer of sugarcane in India is Maharashtra. This state covers around 940,000 hectares of area with sugarcane production. This state produces around 61.32 million tonnes of sugarcane per year on average."), 
             pn.pane.PNG('sugarcane.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_sugar, 
                           sugar_plot.panel(width=700), margin=(0,25)), 
                 pn.Column(yaxis_caneCrushed, caneCrushed_bar_plot.panel(width=600))), 
          pn.Row(pn.Column(area_vs_production_scatterplot.panel(width=600,height=400), margin=(0,25)), 
                 pn.Column(yaxis_production,production_plot.panel(width=700), margin=(0,25)))],
    accent_base_color="#219956",
    header_background="#219956",
)
# template.show()
template.servable();